# Análise por Eventos Mundiais
Este notebook carrega os conjuntos curados exportados para `data/exports/event_windows` e plota, para cada evento, a evolução normalizada por **categoria** (Tecnologia, Agronegócio, Petróleo e Gás, Fundos Imobiliários, Dólar) em janelas de 60 dias: antes, durante e depois.

In [ ]:
import os, json, re
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='whitegrid')
base_export = os.path.join('data', 'exports', 'event_windows')
events_cfg = os.path.join('config', 'events.json')

def slugify(name: str) -> str:
    return re.sub(r'[^a-z0-9_\-]', '-', name.lower().replace(' ', '-'))

def pick_price_column(df: pd.DataFrame) -> str:
    if 'AdjClose' in df.columns: return 'AdjClose'
    if 'Adj Close' in df.columns: return 'Adj Close'
    return 'Close'

def load_window_df(slug: str, window: str) -> pd.DataFrame:
    path = os.path.join(base_export, slug, window)
    if not os.path.exists(path):
        raise FileNotFoundError(f'Pasta não encontrada: {path}. Execute scripts\run_pipeline.ps1 para exportar os dados.')
    parts = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.endswith('.parquet'):
                parts.append(pd.read_parquet(os.path.join(root, f)))
    if not parts:
        raise RuntimeError(f'Nenhum arquivo parquet encontrado em {path}')
    df = pd.concat(parts, ignore_index=True)
    # normaliza nomes de colunas
    df.rename(columns={'Date':'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    return df

def normalize_by_anchor(df: pd.DataFrame, anchor_date: str) -> pd.DataFrame:
    price_col = pick_price_column(df)
    # baseline por categoria no dia da âncora (se faltar, usa o primeiro dia disponível)
    def baseline(group: pd.DataFrame) -> float:
        g = group.sort_values('date')
        anchor_val = g.loc[g['date'] == pd.to_datetime(anchor_date), price_col]
        if len(anchor_val) == 0:
            return g.iloc[0][price_col]
        return anchor_val.iloc[0]
    baselines = df.groupby('category').apply(baseline).rename('baseline').reset_index()
    out = df.merge(baselines, on='category', how='left')
    out['norm_index'] = (out[price_col] / out['baseline']) * 100.0
    return out

def aggregate_category_daily(df: pd.DataFrame) -> pd.DataFrame:
    # média diária por categoria
    return (df.groupby(['category','date'])
              .agg(mean_index=('norm_index','mean'))
              .reset_index())

with open(events_cfg, 'r', encoding='utf-8') as f:
    events = json.load(f)

events[:2]  # inspeção rápida

In [ ]:
# Plota um gráfico por evento, com linhas por categoria
for ev in events:
    name = ev['name']
    anchor = ev['anchor_date']
    slug = slugify(name)

    df_pre = load_window_df(slug, 'pre')
    df_during = load_window_df(slug, 'during')
    df_post = load_window_df(slug, 'post')

    df_all = pd.concat([df_pre, df_during, df_post], ignore_index=True)
    df_all = normalize_by_anchor(df_all, anchor)
    df_cat = aggregate_category_daily(df_all)

    plt.figure(figsize=(10,5))
    sns.lineplot(data=df_cat, x='date', y='mean_index', hue='category')
    plt.axvline(pd.to_datetime(anchor), color='black', linestyle='--', label='Âncora')
    plt.title(f'{name} — Índice normalizado por categoria (Âncora=100)')
    plt.ylabel('Índice (Âncora=100)')
    plt.xlabel('Data')
    plt.legend(title='Categoria')
    plt.tight_layout()
    plt.show()

print('Concluído.')